---

Notebook ini digunakan untuk proses scraping data pada website Amazon

Proses scraping menggunakan bantuan ScraperAPI untuk melakukan scraping website dan Beautiful Soup untuk parsing data

---

# Import Library

Import Library yang dibutuhkan

In [14]:
import pandas as pd
import json
import time
from bs4 import BeautifulSoup
import requests

# Scraping URL Product

Scraping Pertama dilakukan untuk mencari url produk dari pencarian di website Amazon

In [116]:
# Masukkan API-Key untuk ScraperAPI
api_key = "4f2f0c75d5c53c866d86370b14c78fd3"
response=[]

for page in range(1,6):
    """
    Perulangan untuk melakukan scraping sebanyak 5 halaman
    """
    url = "https://www.amazon.com/s?k=huawei+watch&page={page}&xpid=XhyfNtVBb_KDM&crid=TQ7B81M30MY5&qid=1752464026&sprefix=huawei+wa%2Caps%2C388&ref=sr_pg_{page}"

    # Token API
    payload = {
    'api_key': api_key,
    'url': url,
    'country_code': 'us',  
    'render': 'false'      
    }

    # Error Handling agar tidak berhenti saat tidak ada respon
    try:
        res = requests.get("https://api.scraperapi.com", params=payload, timeout=15)
        res.raise_for_status()  
        response.append(res)
        print(f"Sukses mengambil halaman {page}")
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil halaman {page}: {e}")
        continue  
    
    # Time delay
    time.sleep(5)  


Sukses mengambil halaman 1
Sukses mengambil halaman 2
Sukses mengambil halaman 3
Sukses mengambil halaman 4
Sukses mengambil halaman 5


In [ ]:
# Ekstrak link produk
product_url = []

for res in response:
    """
    Perulangan untuk membuka data hasil scraping
    """
    extractor = BeautifulSoup(res.text, 'html.parser')

    # Ambil semua elemen hyperlink
    links = extractor.find_all('a', href=True)
    for link in links:
        href = link['href']

        if (
            "/dp/" in href and
            not href.startswith("/gp/") and
            "aax-us-iad" not in href and
            "slredirect" not in href
        ):
            # Membuat link URL
            base_url = "https://www.amazon.com"
            product_link = href.split('?')[0]  
            full_url = base_url + product_link

            # Hindari duplikat
            if full_url not in product_url:
                product_url.append(full_url)

# Menampilkan hasil link produk
for i, url in enumerate(product_url, start=1):
    print(f"{i}. {url}")

1. https://www.amazon.com/HUAWEI-Smartwatch-Sapphire-Titanium-Compatible/dp/B0F32QWXHV/ref=sr_1_3
2. https://www.amazon.com/HUAWEI-Smartwatch-AI-Powered-Monitoring-Breathing/dp/B0DWFQCY3V/ref=sr_1_4
3. https://www.amazon.com/HUAWEI-Smartwatch-Battery-Tracking-Compatible/dp/B0DDL8SLXN/ref=sr_1_5
4. https://www.amazon.com/Smartwatch-Sharp-Edged-Pro-Level-Tracking-Compatible/dp/B0DDL7Q6PF/ref=sr_1_6
5. https://www.amazon.com/HUAWEI-Design%EF%BC%8CAll-Round-Management%EF%BC%8CComprehensive-Managment%EF%BC%8CDurable-Life%EF%BC%8CCompatible/dp/B0D1CP9WBT/ref=sr_1_8
6. https://www.amazon.com/HUAWEI-Smartwatch-Sharp-Edged-Pro-Level-Compatible/dp/B0DDL729DJ/ref=sr_1_9
7. https://www.amazon.com/HUAWEI-Design%EF%BC%8CAll-Round-Management%EF%BC%8CComprehensive-Managment%EF%BC%8CDurable-Life%EF%BC%8CCompatible/dp/B0D1CM1XSK/ref=sr_1_10
8. https://www.amazon.com/HUAWEI-Smartwatch-Battery-Tracking-Compatible/dp/B0DDL9XQHZ/ref=sr_1_11
9. https://www.amazon.com/HUAWEI-Bluetooth-Smartwatch-AMOLED-Fluoro

# Scraping Data Product

Setelah link untuk setiap produk didapatkan dilakukan scraping kedua untuk mendapatkan data produk

In [111]:
scraping_review = []
for page in product_url:
    """
    Perulangan untuk melakukan scraping terhadap semua link yang ada di list url
    """
    # Token API
    payload = {
    'api_key': api_key,
    'url': page,
    'country_code': 'us',  
    'render': 'true'      
    }

    # Error Handling agar tidak berhenti saat tidak ada respon
    try:
        response = requests.get("https://api.scraperapi.com", params=payload, timeout=15)
        response.raise_for_status()  
        scraping_review.append(response)
        print(f"Sukses: {page}")
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil {page}: {e}")
        continue  
    
    # Time Delay
    time.sleep(5)  


Sukses: https://www.amazon.com/HUAWEI-Smartwatch-Sapphire-Titanium-Compatible/dp/B0F32QWXHV/ref=sr_1_3
Sukses: https://www.amazon.com/HUAWEI-Smartwatch-AI-Powered-Monitoring-Breathing/dp/B0DWFQCY3V/ref=sr_1_4
Gagal mengambil https://www.amazon.com/HUAWEI-Smartwatch-Battery-Tracking-Compatible/dp/B0DDL8SLXN/ref=sr_1_5: HTTPSConnectionPool(host='api.scraperapi.com', port=443): Read timed out. (read timeout=15)
Sukses: https://www.amazon.com/Smartwatch-Sharp-Edged-Pro-Level-Tracking-Compatible/dp/B0DDL7Q6PF/ref=sr_1_6
Sukses: https://www.amazon.com/HUAWEI-Design%EF%BC%8CAll-Round-Management%EF%BC%8CComprehensive-Managment%EF%BC%8CDurable-Life%EF%BC%8CCompatible/dp/B0D1CP9WBT/ref=sr_1_8
Sukses: https://www.amazon.com/HUAWEI-Smartwatch-Sharp-Edged-Pro-Level-Compatible/dp/B0DDL729DJ/ref=sr_1_9
Sukses: https://www.amazon.com/HUAWEI-Design%EF%BC%8CAll-Round-Management%EF%BC%8CComprehensive-Managment%EF%BC%8CDurable-Life%EF%BC%8CCompatible/dp/B0D1CM1XSK/ref=sr_1_10
Sukses: https://www.amazon.co

In [112]:
# List untuk menyimpan data produk
review_data =[]

for product in scraping_review:
    """
    Perulangan untuk melakukan parsing data hasil scraping
    """
    # Memanggil model BS4
    parser = BeautifulSoup(product.text, 'html.parser')

    # Parsing Nama Produk
    title = parser.find(id='productTitle')
    product_title=title.get_text(strip=True) if title else 'N/A'

    # Parsing Gambar produk
    img_tag = parser.select_one('#imgTagWrapperId img')
    image_url = img_tag['src'] if img_tag else 'N/A'

    # Parsing Harga
    price = parser.select_one('.a-price .a-offscreen')
    product_price = price.get_text(strip=True) if price else 'N/A'

    # Parsing Rating Produk
    rating = parser.find('span', {'class': 'a-icon-alt'})
    product_rating=rating.get_text(strip=True) if rating else 'N/A'

    # Parsing Feature
    feature_row = parser.find("tr", {"class": "a-spacing-small po-special_feature"})
    feature_td = feature_row.find("td", class_="a-span9") if feature_row else None
    feature = feature_td.get_text(strip=True) if feature_td else 'N/A'
    
    # Parsing Battery Capacity
    battery_row = parser.find("tr", {"class": "a-spacing-small po-battery.capacity"})
    battery_td = battery_row.find("td", class_="a-span9") if battery_row else None
    battery_cap = battery_td.get_text(strip=True) if battery_td else 'N/A'

    # Parsing Connectivity
    conn_row = parser.find("tr", {"class": "a-spacing-small po-connectivity_technology"})
    comnn_td = conn_row.find("td", class_="a-span9") if conn_row else None
    connectivity = comnn_td.get_text(strip=True) if comnn_td else 'N/A'

    # Parsing fitur GPS
    gps_row = parser.find("tr", {"class": "a-spacing-small po-gps_navigation"})
    gps_td = gps_row.find("td", class_="a-span9") if gps_row else None
    gps = gps_td.get_text(strip=True) if gps_td else 'N/A'
    
    # Parsing Screen Size
    size_row = parser.find("tr", {"class": "a-spacing-small po-display.size"})
    size_td = size_row.find("td", class_="a-span9") if size_row else None
    size = size_td.get_text(strip=True) if size_td else 'N/A'

    # Mengambil data reviews
    reviews = []
    review_blocks = parser.find_all('li', {'data-hook': 'review'})
    for block in review_blocks:

        # Mengambil data rating reviews
        rating_tag = block.find('i', {'data-hook': 'review-star-rating'})
        rating_val = 'N/A'
        if rating_tag:
            rating_span = rating_tag.find('span', class_='a-icon-alt')
            rating_val = rating_span.get_text(strip=True) if rating_span else 'N/A'

        # Mengambil teks reviews
        text_tag = block.find('span', {'data-hook': 'review-body'})
        text_val = 'N/A'
        if text_tag:
            inner_span = text_tag.find('span')
            text_val = inner_span.get_text(strip=True) if inner_span else text_tag.get_text(strip=True)

        # Menyimpan data reviews jika teks dan rating not null
        if text_val != 'N/A' and rating_val != 'N/A':
            reviews.append({
                'review': text_val,
                'rating': rating_val
            })

    # Skip produk tanpa review
    if not reviews:
        continue  

    # Membuat Dictionary review
    dict_review = {
        "title" : product_title,
        "rating" : product_rating,
        "price" : product_price,
        "features" : feature,
        "Battery Capacity" : battery_cap,
        "Connectivity": connectivity,
        "GPS" : gps,
        "Screen Size": size,
        "img_url": image_url,
        "Brand": "Huawei",
        "Review" : reviews
    }

    # Menyimpan data review produk ke dalam list
    review_data.append(dict_review)


In [113]:
# Menghapus data tanpa title
review_data = [item for item in review_data if item["title"] != "N/A"]
review_data

[{'title': 'HUAWEI Watch FIT4 Pro Smartwatch, 1.82" Sapphire Display Titanium Alloy Bezel, Up to 10-Day Battery Life, Dual Band GPS, Golf Courses map, 40 m Free Diving, Compatible with Android iOS, Black',
  'rating': '4.4 out of 5 stars',
  'price': '$293.67',
  'features': 'Touchscreen, Lightweight, Multisport Tracker, Activity Tracker, Always On Display',
  'Battery Capacity': '400 Milliamp Hours',
  'Connectivity': 'Bluetooth',
  'GPS': 'Built-in GPS',
  'Screen Size': '1.82 Inches',
  'img_url': 'https://m.media-amazon.com/images/I/61jvoRsvS8L.__AC_SX300_SY300_QL70_FMwebp_.jpg',
  'Brand': 'Huawei',
  'Review': [{'review': 'These HUAWEI Watch FIT4 Pro smartwatches are surprisingly comfortable to wear all day.The battery life is really good, even with regular use and GPS tracking.I like how light they feel on the wrist and the screen is bright and clear. The titanium bezel looks and feels premium.Overall very happy with this purchase. Great mix of style, features and performance.',

# Saving Data

Setelah data produk didapatkan, data disimpan dalam format JSON karena terdapat list dictionary yang berisi data reviews

In [114]:
# Menyimpan data dalam format JSON
with open("huawei.json", "w", encoding="utf-8") as review_json:
    json.dump(review_data, review_json, indent=2, ensure_ascii=True)